In [1]:
from abc import ABC, abstractmethod
import os
from dotenv import load_dotenv
from sqlalchemy.orm import scoped_session, Session
import sys
sys.path.append('..')
from src.repo.orm import OpenPool, PrayerRequestORM
import tqdm
load_dotenv()
pg_uri = os.environ.get('PRAYERS_PG_DATABASE_URL')

pool: scoped_session[Session] = OpenPool(pg_uri)

class Backill(ABC):
    @abstractmethod
    def filter(self, session: Session)->list[PrayerRequestORM]:
        pass

    @abstractmethod
    def update_fields(self, ormRequest: PrayerRequestORM):
        pass

# Backfill the prayer request table
# Use tqdm to show progress
def PrayerRequestBackfill(backfill: Backill):
    with pool() as session:
        results = backfill.filter(session)
        for ormRequest in tqdm.tqdm(results):
            backfill.update_fields(ormRequest)
        session.commit()
        session.close()

In [3]:
from src.models.models import ClassifierModels

classifiers = ClassifierModels()

class BackfillSentiment(Backill):
    def filter(self, session: Session)->list[PrayerRequestORM]:
        return session.query(PrayerRequestORM).filter(
            PrayerRequestORM.sentiment_analysis == None ).all()

    def update_fields(self, ormRequest: PrayerRequestORM):
        result = classifiers.classify(ormRequest.request)
        ormRequest.sentiment_analysis = result['sentiment']
        ormRequest.emotion_roberta = result['emotion']

backfiller = BackfillSentiment()
# PrayerRequestBackfill(backfiller)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 778/778 [03:53<00:00,  3.33it/s]
